In [26]:
# import ultralytics
# from ultralytics import YOLO
from ultralytics import YOLO
import cv2
import numpy as np
from roboflow import Roboflow

from pathlib import Path
import random
import shutil
import torch

# download dataset

In [2]:
# current path print 
Path.cwd().resolve()

PosixPath('/Users/anassyed/Documents/lynqo/notebooks')

In [3]:
PATH_TO_DADASET = Path("../datasets/Surgical-Tools-Detection-1")
if not PATH_TO_DADASET.exists():
    rf = Roboflow(api_key="P9o0JjFVWdK2Z3nzefUI")
    project = rf.workspace("northeastern-university-yu4fz").project("surgical-tools-detection-c8b3w")
    version = project.version(1)
    dataset = version.download("yolov11")
else:
    print("Dataset folder already exists, skipping download.")
                

Dataset folder already exists, skipping download.


In [71]:
# getting the instrument names from config.json 
config_path = PATH_TO_DADASET / "config.json"
with open(config_path, 'r') as f:
    config = f.read()
    config = eval(config)  # Convert string to dictionary
    instrument_names = config['SURGICAL_INSTRUMENTS']
    print("Instrument names:", instrument_names)


Instrument names: {'0': 'Mayo-Hegar needle holder', '1': 'Adson tissue forceps (plain thumb forceps)', '2': 'Sponge-holding (Foerster) forceps', '3': 'Kelly hemostat (straight)', '4': 'DeBakey tissue forceps', '5': 'Mosquito hemostat (straight)', '6': 'Weitlaner self-retaining retractor', '7': 'Surgical scissors (e.g. Metzenbaum/Mayo scissors)', '8': 'Kelly hemostat (curved)', '9': 'Mixter (right-angle) hemostat', '10': 'Bone hook', '11': 'Lister (bandage) scissors', '12': 'Tissue forceps (dressing forceps)', '13': 'Kocher (Ochsner) forceps', '14': 'Frazier suction tip', '15': 'Electrocautery ("Bovie") pencil', '16': 'Scalpel (No. 3 handle with blade)', '17': 'Kelly hemostat (long)'}


After augmentation

In [14]:
# Setup paths for original and augmented data
train_img_dir = PATH_TO_DADASET / 'train' / 'images'
train_aug_dir = PATH_TO_DADASET / 'train' / 'images_aug'
train_lbl_dir = PATH_TO_DADASET / 'train' / 'labels'
train_lbl_aug_dir = PATH_TO_DADASET / 'train' / 'labels_aug'

# Get all images (both original and augmented)
all_images = list(train_img_dir.glob('*.jpg'))
all_aug_images = list(train_aug_dir.glob('*.jpg'))
total_images = len(all_images) + len(all_aug_images)
subset_size = int(total_images * 0.25)  # 25% of total

print(f"Original images: {len(all_images)}")
print(f"Augmented images: {len(all_aug_images)}")
print(f"Total images: {total_images}")


Original images: 17789
Augmented images: 35417
Total images: 53206


In [17]:
# Create subset directories
subset_size = int(total_images * 0.10) # 10% of total
subset_dir = PATH_TO_DADASET / 'train_subset'
subset_img_dir = subset_dir / 'images'
subset_lbl_dir = subset_dir / 'labels'

subset_img_dir.mkdir(parents=True, exist_ok=True)
subset_lbl_dir.mkdir(parents=True, exist_ok=True)

# Randomly select from both original and augmented
random.seed(42)  # for reproducibility
combined_images = all_images + all_aug_images
selected_images = random.sample(combined_images, subset_size)

In [18]:
# Copy selected files
for img_path in selected_images:
    # Copy image
    shutil.copy2(img_path, subset_img_dir)
    
    # Determine correct label directory based on image source
    if 'images_aug' in str(img_path):
        lbl_dir = train_lbl_aug_dir
    else:
        lbl_dir = train_lbl_dir
        
    # Copy corresponding label
    lbl_path = lbl_dir / f"{img_path.stem}.txt"
    if lbl_path.exists():
        shutil.copy2(lbl_path, subset_lbl_dir)

print(f"\nCreated subset dataset at: {subset_dir}")


Created subset dataset at: ../datasets/Surgical-Tools-Detection-1/train_subset


In [19]:
# Create modified yaml for subset training
import yaml

yaml_path = PATH_TO_DADASET / 'data.yaml'
subset_yaml_path = PATH_TO_DADASET / 'data_subset.yaml'

with open(yaml_path) as f:
    yaml_data = yaml.safe_load(f)

# Update train path to use subset
yaml_data['train'] = str(subset_img_dir.relative_to(PATH_TO_DADASET))

with open(subset_yaml_path, 'w') as f:
    yaml.dump(yaml_data, f)

In [20]:
# Initialize YOLO model for training
model = YOLO("yolo11n.pt")


In [24]:
use_subset = True  # Set to True to use the subset dataset
if use_subset:
    data_path = subset_yaml_path
else:
    data_path = yaml_path

In [25]:

# Train the model using the dataset
model.train(
    data=str(data_path),  # path to data yaml file using PATH_TO_DADASET
    epochs=5,                # number of epochs
    imgsz=416,                # image size
    batch=16,                 # batch size
    patience=50,              # early stopping patience
    verbose=True             # print training progress
)

Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.7.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/Surgical-Tools-Detection-1/data_subset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pret

train: Scanning /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/labels... 5320 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5320/5320 [00:00<00:00, 6854.49it/s]

train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00295_jpg.rf.a6fad8291ea70bd9ffd28b1e51de1838_aug0.jpg: 1 duplicate labels removed
train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00519_jpg.rf.7d55bd7f4c6161616bb9ff9c81173677_aug0.jpg: 1 duplicate labels removed
train: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/images/Add_labelled00695_jpg.rf.4e8710dc23e98c473cc20402c9eccd1c_aug1.jpg: 1 duplicate labels removed


train: New cache created: /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/train_subset/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 55.9±50.7 MB/s, size: 10.1 KB)


val: Scanning /Users/anassyed/Documents/lynqo/datasets/Surgical-Tools-Detection-1/valid/labels.cache... 1980 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1980/1980 [00:00<?, ?it/s]


...

                   all       1980       2551      0.242      0.526      0.285      0.235



5 epochs completed in 1.849 hours.
Optimizer stripped from /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/last.pt, 5.4MB
Optimizer stripped from /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/best.pt, 5.4MB

Validating /Users/anassyed/Documents/lynqo/runs/detect/train9/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.11 torch-2.7.0 CPU (Apple M2)
YOLO11n summary (fused): 100 layers, 2,585,662 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 62/62 [02:08<00:00,  2.07s/it]


                   all       1980       2551      0.241      0.528      0.284      0.235
                     0        176        195      0.206      0.769      0.287      0.261
                     1         74        102      0.231      0.549      0.211      0.172
                    10         92        105      0.446      0.486       0.48       0.33
                    11        172        197      0.312      0.772      0.695      0.575
                    12        100        127      0.166     0.0866      0.187      0.157
                    13        109        117      0.223      0.735      0.337      0.298
                    14         96        113      0.188      0.469      0.144      0.129
                    15        107        107      0.402      0.991      0.511      0.347
                    16         80         80      0.266          1      0.339      0.248
                    17         94         94      0.308      0.362      0.266      0.256
                     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x31ca9e990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.0

In [30]:
# Load YOLO model
MODEL_PATH = Path("../runs/detect/train9/weights/best.pt")
# loaded_model = YOLO(MODEL_PATH)
loaded_model = YOLO(MODEL_PATH)

In [32]:
results = loaded_model.predict(
    source=str(subset_img_dir),  # path to the images directory
    conf=0.25,                   # confidence threshold
    save=True,                   # save results
    save_txt=True,               # save results in txt format
    save_conf=True,              # save confidence scores
    show=True                    # display results
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/5320 /Users/anassyed/Documents/lynqo/notebooks/../datasets/Surgical-Tools-Detection-1/train_subset/images/250365762_633967454294269_8117536681550006933_n_jpg.rf.425dc6c6b0a4aa8541e3350e3ecd2c7b.jpg: 416x416 (no detections), 51.8ms
image 2/5320 /Users/anassyed/Documents/lynqo/notebooks/../datasets/Surgical-Tools-Detection-1/train_subset/images/250365762_633967454294269_8117536681550006933_n_jpg.rf.6bd9dd31c66c1e5591a816b281d7c213_aug1.jpg: 416x4

KeyboardInterrupt: 

In [ ]:

# Display results
for result in results:
    print(f"Image: {result.path}")
    for box in result.boxes:
        print(f"  Class: {box.cls}, Confidence: {box.conf:.2f}, BBox: {box.xyxy}")  # xyxy format
        # If you want to visualize the bounding boxes, you can use OpenCV or matplotlib here
        # For example, using OpenCV:
        img = cv2.imread(result.path)
        cv2.rectangle(img, (int(box.xyxy[0]), int(box.xyxy[1])), (int(box.xyxy[2]), int(box.xyxy[3])), (255, 0, 0), 2)
        cv2.putText(img, f"{box.cls} {box.conf:.2f}", (int(box.xyxy[0]), int(box.xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.imshow("Result", img)
        cv2.waitKey(0)  # Wait for a key press to close the image window
cv2.destroyAllWindows()  # Close all OpenCV windows
# Save the model

In [45]:
results[0].boxes[0]

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([13.])
conf: tensor([0.1371])
data: tensor([[6.9766e+02, 2.6796e+02, 8.9964e+02, 4.5225e+02, 1.3706e-01, 1.3000e+01]])
id: None
is_track: False
orig_shape: (580, 900)
shape: torch.Size([1, 6])
xywh: tensor([[798.6515, 360.1069, 201.9794, 184.2959]])
xywhn: tensor([[0.8874, 0.6209, 0.2244, 0.3178]])
xyxy: tensor([[697.6618, 267.9590, 899.6412, 452.2549]])
xyxyn: tensor([[0.7752, 0.4620, 0.9996, 0.7797]])

In [51]:
def predict_image(model, image_path, conf_threshold=0.1, save=True, show=True):
    """
    Perform prediction on an image using a YOLO model.
    
    Args:
        model: YOLO model for prediction
        image_path (str): Path to the input image
        conf_threshold (float): Confidence threshold for detections
        save (bool): Whether to save the results
        show (bool): Whether to display the results
    
    Returns:
        results: YOLO prediction results
    """
    results = model.predict(
        source=image_path,
        conf=conf_threshold,
        save=save,
        save_txt=True,
        save_conf=True,
        show=show
    )
    return results

In [49]:
def display_detections(results, model):
    """
    Display detected objects with their class names and confidence scores.
    
    Args:
        results: YOLO detection results
        model: YOLO model with class names mapping
    """
    try:
        # Get the names dictionary from the model
        names = model.names
        
        for r in results:
            print(f"\nDetections in {Path(r.path).name}:")
            
            if not hasattr(r, 'boxes') or len(r.boxes) == 0:
                print("No detections found.")
                continue
                
            for box in r.boxes:
                # Get the class index and convert to class name
                class_id = int(box.cls[0])
                class_name = names.get(class_id, "Unknown")
                confidence = float(box.conf[0])
                
                print(f"- {class_name} (Confidence: {confidence:.2f})")
                
    except Exception as e:
        print(f"Error processing detections: {str(e)}")

In [82]:
result = predict_image(loaded_model, 
              '../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg')


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg: 320x416 1 11, 1 2, 41.1ms
Speed: 2.6ms preprocess, 41.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict
66 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict/labels
image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg: 320x416 1 11, 1 2, 41.1ms
Speed: 2.6ms preprocess, 41.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict
66 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict/labels


In [83]:
display_surgical_detections(result)


Detected Surgical Instruments in stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg:
--------------------------------------------------
Instrument: Bone hook
Confidence: 13.54%
Location (x1,y1,x2,y2): [370.6, 485.2, 986.2, 789.5]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 13.00%
Location (x1,y1,x2,y2): [411.2, 493.0, 938.7, 888.9]
--------------------------------------------------


In [84]:
result = predict_image(loaded_model,
              '../datasets/General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg',
              conf_threshold=0.1, save=True, show=True)


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg: 320x416 1 5, 36.9ms
Speed: 2.1ms preprocess, 36.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict
66 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict/labels
image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg: 320x416 1 5, 36.9ms
Speed: 2.1ms preprocess, 36.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict
66 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict/labels


In [85]:
# Test the new function with surgical instrument names
print("Displaying detections with proper instrument names:")
display_surgical_detections(result)

Displaying detections with proper instrument names:

Detected Surgical Instruments in General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg:
--------------------------------------------------
Instrument: Kocher (Ochsner) forceps
Confidence: 10.22%
Location (x1,y1,x2,y2): [1127.7, 270.7, 1597.7, 872.9]
--------------------------------------------------


In [86]:
def display_surgical_detections(results, SURGICAL_INSTRUMENTS=instrument_names):
    """
    Display detected surgical instruments with their proper names, counts, and confidence scores.
    
    Args:
        results: YOLO detection results
    """
    try:
        for r in results:
            print(f"\nDetected Surgical Instruments in {Path(r.path).name}:")
            print("-" * 50)
            
            if not hasattr(r, 'boxes') or len(r.boxes) == 0:
                print("No instruments detected.")
                continue
            
            # Dictionary to store counts of each instrument
            instrument_counts = {}
            # List to store all detections
            detections = []
            
            # Process all detections
            for box in r.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                instrument_name = SURGICAL_INSTRUMENTS.get(str(class_id), f"Unknown Instrument (Class {class_id})")
                
                # Update counts
                instrument_counts[instrument_name] = instrument_counts.get(instrument_name, 0) + 1
                detections.append((instrument_name, confidence, box.xyxy[0].tolist()))
            
            # First, display summary of counts
            print("Summary of Detected Instruments:")
            for instrument, count in instrument_counts.items():
                print(f"{instrument}: {count} instance{'s' if count > 1 else ''}")
            print("-" * 50)
            
            # Then show detailed information for each detection
            print("\nDetailed Detection Information:")
            # Sort by confidence (highest first)
            detections.sort(key=lambda x: x[1], reverse=True)
            
            for instrument_name, confidence, bbox in detections:
                print(f"Instrument: {instrument_name}")
                print(f"Confidence: {confidence:.2%}")
                print(f"Location (x1,y1,x2,y2): [{bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f}]")
                print("-" * 50)
                
    except Exception as e:
        print(f"Error processing detections: {str(e)}")

In [87]:
# Test the updated function with proper instrument counting
print("Displaying detections with instrument counts:")
display_surgical_detections(result)

Displaying detections with instrument counts:

Detected Surgical Instruments in General-Surgical-Instruments-What-Are-They-And-What-Are-They-Used-For-Medilogic-21564597.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Kocher (Ochsner) forceps: 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Kocher (Ochsner) forceps
Confidence: 10.22%
Location (x1,y1,x2,y2): [1127.7, 270.7, 1597.7, 872.9]
--------------------------------------------------


In [88]:
# Test with another image
result = predict_image(loaded_model,
              '../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg',
              conf_threshold=0.1, save=True, show=True)

# Display the results
print("\nDisplaying detections for new image:")
display_surgical_detections(result)


image 1/1 /Users/anassyed/Documents/lynqo/notebooks/../datasets/stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg: 320x416 1 11, 1 2, 105.2ms
Speed: 5.4ms preprocess, 105.2ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 416)
Results saved to /Users/anassyed/Documents/lynqo/runs/detect/predict
66 labels saved to /Users/anassyed/Documents/lynqo/runs/detect/predict/labels

Displaying detections for new image:

Detected Surgical Instruments in stock-photo-surgical-instruments-in-instrument-tray-731922088.jpg:
--------------------------------------------------
Summary of Detected Instruments:
Bone hook: 1 instance
Kelly hemostat (straight): 1 instance
--------------------------------------------------

Detailed Detection Information:
Instrument: Bone hook
Confidence: 13.54%
Location (x1,y1,x2,y2): [370.6, 485.2, 986.2, 789.5]
--------------------------------------------------
Instrument: Kelly hemostat (straight)
Confidence: 13.00%
Location (x1,y1,x2,y2): [41